In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

from sklearn.decomposition import PCA

In [ ]:
## Paths Input Here
analysis = "principal_component_analysis/5mm_maxima_v2"
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/final_analyses/maxima_and_minima_rois/roi_roi_correlation/maxima_to_vta/matrix_corrMx_T (10)_revised_with_clinical_data_dataframe.csv'
    clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/patient_data/AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = os.path.join(os.path.dirname(conn_path), f'{analysis}')
    #out_dir = r'path to out dir here'
    x_roi_names = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/rios_ferguson_yeo_and_conjunction_cluster_roi_roi_correlation/matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Mac style')
    print('I will save to:', out_dir)
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\AD_to_memory_net'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')

if os.path.isdir(out_dir) != True:
    os.makedirs(out_dir)

save = True
do_transform = False

## Option A -- Import Pre-prepared Dataframe
Learn relationships ROIs at the ROI-ROI correl level

In [ ]:
data_df = pd.read_csv(conn_path)

#Eliminate Columns of no Interest
try:
    data_df.pop('Unnamed: 0')
    data_df.pop('Randomization Arm')
    data_df.pop('Patient # CDR, ADAS')
    data_df.pop('Age at DOS')
    data_df.pop('Baseline CDR (sum of squares)')
    data_df.pop('% Change from baseline (CDR)')
    data_df.pop('Baseline ADAS-Cog11')    
except:
    print('Could not find ("Unnamed: 0") ')
#Rename variables to prevent errors


#----------------------------------------------------------------Edit Dataframe----------------------------------------------------------------
#Set a list of the variables to be fed into DF
# change_string = ''
# for i in range (0, len(variables_to_change['original'])):
#     change_string += f"variables_to_change['original'][{i}]: variables_to_change['destination'][{i}],"
# change_string = 'data_df.rename(columns={' + change_string[0:-1] + '})'
# data_df = eval(change_string)

#Organize the columns
# import natsort
# natsorted_columns = natsort.natsorted(data_df.columns)
# data_df = data_df.reindex(columns=natsorted_columns)

#Place response column at start of dataframe
# response_series = data_df.pop(variables_to_change['destination'][0])
# data_df.insert(0, variables_to_change['destination'][0], response_series)


#Remove outlier
# outlier_index=[9, 47, 48, 49]
# data_df = data_df.drop(index=outlier_index)

#Standardize the data
preserved_df = data_df.copy()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
#Remove anything you don't want to standardize
# age_df = data_df.pop('age')
data_df.iloc[:,:] = scaler.fit_transform(data_df.iloc[:,:].values)
#reinster things to standardize
# data_df['age'] = age_df

#Handle NaNs
#Drop NANs
data_df.dropna(inplace=True)

display(data_df)

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

def pca_function(dataframe, dataframe_indices, num_components=None):
    
    # Define the PCA class/method
    pca = PCA(n_components=num_components)
    x_vals = dataframe.loc[:, dataframe_indices].to_numpy()
    pca.fit(x_vals)

    # Transform the independent variables into their principal components
    pca_vals = pca.transform(x_vals)

    # Create a dataframe from the PCA
    pca_df = pd.DataFrame(pca_vals)

    # Rename the column names
    colnames = []
    for col in pca_df.columns:
        colnames.append('Component_'+str(col+1))
    pca_df.columns = colnames
    
    # Extract the weights of the principal components
    weights = pd.DataFrame(pca.components_, columns=dataframe.loc[:, dataframe_indices].columns.values.tolist())
    weights.index = pca_df.columns

    return pca_df, pca, weights

In [ ]:
dataframe_indices = pd.IndexSlice['ferguson_memnet_ant_med_thalamus_left':]
pca_df, pca, weights = pca_function(data_df, dataframe_indices)

print('Principal Components Below')
display(pca_df)

In [ ]:
#Get weights from the PCA
print('Weights of Principal Components Below')
display(weights)

In [ ]:
from calvin_utils.palm import brain_permutation
def permute_pca_weights(df_to_permute, n_permutations, num_components=None):
    '''
    This function receives only a dataframe of things to permute and perform a PCA on
    It then calculates the number of 
    
    Inputs: 
    df_to_permute - a dataframe of things to permute with no excess rows or columns
    n_permutations - number of permutations, simply an integer
    
    Outputs:
    permuted_results - a dataframe of permuted weights for each permutation
    '''
    #initialize array to hold permuted results
    permuted_weights = np.zeros((n_permutations, df_to_permute.shape[1]))
    
    for i in range(0, n_permutations):
        #Permute the input dataframe
        
        #Permute Rows
        for i in range(0, df_to_permute.shape[0]):
            df_to_permute.iloc[i,:] = np.random.permutation(df_to_permute.iloc[i,:])
        #Permute columns
        for i in range(0, df_to_permute.shape[1]):
            df_to_permute.iloc[:,i] = np.random.permutation(df_to_permute.iloc[:,i])
        # permuted_completed_permutation = np.apply_along_axis(lambda x: np.random.permutation(x), 1, df_to_permute.values)
        permuted_df = df_to_permute
        
        #calculate the weights
        dataframe_indices = pd.IndexSlice[:]
        pca_df, fitted_pca, weights = pca_function(permuted_df, dataframe_indices, num_components=num_components)
        permuted_weights[i,:] = weights.iloc[0,:]
    return permuted_weights

def calculate_empiric_p_value(observed_df, permuted_df):
    permutations_occuring_above_observations = np.where(permuted_df > observed_df, 1, 0)
    
    num_occurences_above_observations = np.count_nonzero(permutations_occuring_above_observations)
    num_permutations = permuted_df.shape[0]
    
    empiric_p_value = num_occurences_above_observations/num_permutations
    
    return empiric_p_value

In [ ]:
#Permute the PCA to Analyze the Significance of the Weights
from calvin_utils.palm import permute_contrast_matrix
permuted_weights = permute_pca_weights(data_df.loc[:,dataframe_indices], 10000, None)
permuted_weights = pd.DataFrame(permuted_weights, columns=weights.columns)

In [ ]:
#Calculate the p values
p_value = []
for i in range(0, len(weights.columns)):
    p_value.append(calculate_empiric_p_value(weights.iloc[0,i], permuted_weights.iloc[:,i].values))
p_values_df = pd.DataFrame(p_value, weights.columns).transpose()
display(p_values_df)

In [ ]:
#Perform PCA
#Merge original and PCA dataframe?
merge = True
##--------------------------------USER INPUT------------------------------------------------

#Merge dataframes
if merge:
    from calvin_utils.merge_dataframes import merge_dataframes
    merged_df = merge_dataframes(data_df.iloc[:, 0:1], pca_df)
else:
    merged_df = pca_df
display(merged_df)
#Merge the dataframe back to the orignal
if save:
    merged_df.to_csv(os.path.join(out_dir, 'pca_with_clinical_data.csv'))
    weights.to_csv(os.path.join(out_dir, 'pca_weights.csv'))

In [ ]:
# Explained Variance Plot
# Calculate the explained variance for each component
explained_variance = pca.explained_variance_ratio_
print(explained_variance)
sns.set_style('white')
sns.set_palette('Greys', 1, desat=1)
# Plot the explained variance by each component
fig = plt.figure(figsize=(8,8))
plt.plot(range(len(explained_variance)), explained_variance)
plt.xlabel('Component')
plt.ylabel('Explained Variance')
plt.show()

if save:
    fig.savefig(os.path.join(out_dir, 'explained_variance_plot.png'))

In [ ]:
# Scree Plot
# Plot the Scree Plot
fig = plt.figure(figsize=(8,8))
plt.plot(range(len(explained_variance)), np.cumsum(explained_variance))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.show()


if save:
    fig.savefig(os.path.join(out_dir, 'scree_plot.png'))


## Perform Linear Modelling of PCA to Outcomes

In [ ]:
import statsmodels.formula.api as smf
# merged_df = merged_df.rename(columns={'% Change from baseline (ADAS-Cog11)': 'perc_improvement'})
results = smf.ols('perc_improvement ~ Component_1 + Component_2 + Component_3', data=merged_df).fit()
print(results.summary2())

## Generate a Map From PCA of the Networks

In [ ]:
# Perform New PCA Using Fit PCA
## must use RFz maps and z score them (treat in same manner as PCA ROI-ROI analysis)
from calvin_utils.z_score_matrix import z_score_matrix
from calvin_utils.import_matrices import import_matrices_from_folder

yeo_import = import_matrices_from_folder('/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/seed_generated_networks/yeo_r_maps/r_fz_map', file_pattern='/*.nii*')

#z_score the networks
yeo_networks = yeo_import.copy()
for i in range(len(yeo_import.columns)):
    yeo_networks.iloc[:,i] = z_score_matrix(yeo_import.iloc[:,i])

#Set areas outside of the brain back to zero
yeo_networks[yeo_import == 0] = 0

display(yeo_networks)

In [ ]:
#Isolate the network
yeo_networks = yeo_networks.iloc[:, [1, 2, 3]]
display(yeo_networks)

In [ ]:
import tqdm
from calvin_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
from nilearn import plotting

#Itialize the holding info
pc_1 = []
#Iterate over each voxel, considering the voxel at a given location across each network
for i in tqdm.tqdm(range(0, len(yeo_networks))):
    #Only operate on nonzero voxels
    if np.sum(yeo_networks.iloc[i,:]) != 0:
        #Perform PCA at that voxel based on previously learned PCA
        pc_1.append(pca.transform(yeo_networks.iloc[i,:].to_numpy().reshape(1,-1))[0,0])
    #If voxel=zero, just assign zero
    else:
        pc_1.append(0)
#Place in dataframe
print(np.max(pc_1))
print(np.min(pc_1))
pc1_df = pd.DataFrame(pc_1)

#Save and initialize the nifti
img = nifti_from_matrix(matrix=pc1_df, output_file=f'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/principal_component_topology/{analysis}')

#Display nifti
mask = nimds.get_img("mni_icbm152")
ovr_html1 = plotting.view_img(img, cut_coords=(0,0,0), title=(f'pc_1_map'), black_bg=False, opacity=.75, cmap='ocean_hot')
ovr_html1

## Use the PC Nifti to Generate a Response Topography

In [ ]:
#Create a dataframe to use
var_one = 'Component_1'
input_df = pd.DataFrame({var_one: pc_1})

In [ ]:
import tqdm
#Generate a new matrix from the input dataframe
responses = []

#Work on a voxel-wise basis by iterating over index
for i in tqdm.tqdm(range(0, len(input_df))):
    #Do not calculate on zero values as intercept will be applied in the regressoin
    if np.sum(input_df.iloc[i,0]) != 0:
        #Assign a temporary dataframe with values that the statsmodels model is expecting
        temp_df = pd.DataFrame({var_one: input_df.iloc[i,0]}, index=['temp_vals'])
        #Calculate the voxelwise predicted outcome at a given voxel
        responses.append(results.predict(temp_df)[0])
    else:
        #If voxel is zero-connectivity, assign zero so as to avoid application of intercept
        responses.append(0)
        
#Store responses in a dataframe
response_df = pd.DataFrame()
response_df['response_topology'] = responses
#set values outside of brain back to zero
response_df.loc[:, 'response_topology'][input_df.iloc[:,0] == 0] = 0

#Display the figure
img = nifti_from_matrix(response_df, output_file='/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/response_topology/yeo_pc1_response_topology')

mask = nimds.get_img("mni_icbm152")
ovr_html1 = plotting.view_img(img, cut_coords=(0,0,0), title=(f'conjunction'), black_bg=False, opacity=.75, cmap='ocean_hot')
ovr_html1